In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
import time
from skopt import gp_minimize
from skopt.space import Real, Integer
import abc
import matplotlib.patches as patches

In [2]:
class LatticeBase(abc.ABC):
    def __init__(self, size):
        self.size = size
        self.lattice = None

    @abc.abstractmethod
    def initialize(self):
        pass

    @abc.abstractmethod
    def get_random_site(self):
        pass

    @abc.abstractmethod
    def get_neighbors(self, site):
        pass

    @abc.abstractmethod
    def get_occupancy(self):
        pass

    @abc.abstractmethod
    def plot(self, title="Estado de la red", figsize=(6,6)):
        pass


In [3]:
class SquareLattice(LatticeBase):
    def __init__(self, size):
        super().__init__(size)
        self.lattice = np.zeros((size, size), dtype=int)

    def initialize(self):
        self.lattice.fill(0)

    def get_random_site(self):
        i, j = np.random.randint(0, self.size, size=2)
        return (i, j)

    def get_neighbors(self, site):
        i, j = site
        return [(ni, nj) for ni, nj in [(i-1,j), (i+1,j), (i,j-1), (i,j+1)]
                if 0 <= ni < self.size and 0 <= nj < self.size]

    def get_occupancy(self):
        return np.sum(self.lattice == 1) / (self.size ** 2)

    def plot(self, title="Red cuadrada", figsize=(6,6)):
        fig, ax = plt.subplots(figsize=figsize)
        for i in range(self.size):
            for j in range(self.size):
                color = "black" if self.lattice[i, j] == 1 else "white"
                rect = patches.Rectangle((j, self.size - i - 1), 1, 1, edgecolor='gray', facecolor=color)
                ax.add_patch(rect)
        ax.set_xlim(0, self.size)
        ax.set_ylim(0, self.size)
        ax.set_title(title)
        ax.axis('off')
        plt.show()


In [4]:
import matplotlib.patches as patches
import numpy as np

class HexagonalLattice(LatticeBase):
    def __init__(self, size):
        super().__init__(size)
        self.lattice = np.zeros((size, size), dtype=int)

    def initialize(self):
        self.lattice.fill(0)

    def get_random_site(self):
        i, j = np.random.randint(0, self.size, size=2)
        return (i, j)

    def get_neighbors(self, site):
        i, j = site

        # Definición de vecinos para layout even-r (fila par desplazada)
        if i % 2 == 0:  # Fila par
            deltas = [(-1, 0), (-1, -1),
                    (0, -1),  (0, +1),
                    (+1, 0), (+1, -1)]
        else:  # Fila impar
            deltas = [(-1, +1), (-1, 0),
                    (0, -1),  (0, +1),
                    (+1, +1), (+1, 0)]

        neighbors = [(i + di, j + dj) for di, dj in deltas
                    if 0 <= i + di < self.size and 0 <= j + dj < self.size]

        return neighbors

    def get_occupancy(self):
        return np.sum(self.lattice == 1) / (self.size ** 2)

    def plot(self, title="Red hexagonal", figsize=(6, 6)):
        import matplotlib.pyplot as plt
        import matplotlib.patches as patches

        fig, ax = plt.subplots(figsize=figsize)

        dx = 3 ** 0.5  # horizontal spacing
        dy = 1.5       # vertical spacing
        radius = 0.5

        for i in range(self.size):
            for j in range(self.size):
                x = dx * j + (dx / 2 if i % 2 else 0)
                y = dy * i
                color = "black" if self.lattice[i, j] == 1 else "white"

                hexagon = patches.RegularPolygon(
                    (x, y),
                    numVertices=6,
                    radius=radius,
                    orientation=np.radians(30),
                    edgecolor='gray',
                    facecolor=color,
                    linewidth=0.5
                )
                ax.add_patch(hexagon)

        ax.set_aspect('equal')
        ax.set_xlim(-1, dx * self.size + 1)
        ax.set_ylim(-1, dy * self.size + 1)
        ax.set_title(title)
        ax.axis('off')
        plt.tight_layout()
        plt.show()




In [5]:
from IPython.display import clear_output

class KineticMonteCarlo:
    def __init__(self, lattice, T, processes, time_array, time_factor=1e-7):
        self.lattice = lattice
        self.T = T
        self.processes = processes
        self.k_b = 8.617e-5
        self.h = 4.136e-15
        self.k_0 = 5
        self.factor = time_factor
        self.time_array = time_array
        self.time = time_array[0]

        self.occupancy = {t: None for t in time_array}
        self.events_count = {p: 0 for p in processes}
        self.events_history = {p: [] for p in processes}
        self.times, self.energies = [], []
        self.lattices = []

        self.lattice.initialize()

    # =======================
    #   Transiciones y tasas
    # =======================

    def pick_random_site(self):
        return self.lattice.get_random_site()

    def count_occupied_neighbors(self, site):
        return sum(self.lattice.lattice[n] for n in self.lattice.get_neighbors(site))

    def calculate_transition_rates(self, site):
        n_ij = self.count_occupied_neighbors(site)
        rates = {}

        if "adsorption" in self.processes:
            p = self.processes["adsorption"]
            E_ads = p["e_ads"] + p.get("e_int_abs", 0) * n_ij
            rates["adsorption"] = (self.k_0 * self.k_b * self.T) / self.h * np.exp(-E_ads / (self.k_b * self.T))

        if "desorption" in self.processes:
            p = self.processes["desorption"]
            E_des = p["e_des"] + p.get("e_int_des", 0) * n_ij
            rates["desorption"] = (self.k_0 * self.k_b * self.T) / self.h * np.exp(-E_des / (self.k_b * self.T))

        if "diffusion" in self.processes:
            p = self.processes["diffusion"]
            E_diff = p["e_diff"] + p.get("e_int_diff", 0) * n_ij
            rates["diffusion"] = (self.k_0 * self.k_b * self.T) / self.h * np.exp(-E_diff / (self.k_b * self.T))

        return rates

    def select_event(self, rates):
        total = sum(rates.values())
        r = np.random.rand() * total
        cumulative = 0
        for event, rate in rates.items():
            cumulative += rate
            if r < cumulative:
                return event
        return None

    # ===================
    #     Energía
    # ===================

    def calculate_total_energy(self):
        total_energy = 0
        for i in range(self.lattice.size):
            for j in range(self.lattice.size):
                if self.lattice.lattice[i, j] == 1:
                    n_ij = self.count_occupied_neighbors((i, j))
                    e_ads = self.processes["adsorption"]["e_ads"]
                    e_int_abs = self.processes["adsorption"].get("e_int_abs", 0)
                    total_energy += e_ads + e_int_abs * n_ij
        return total_energy

    # ===================
    #     Simulación
    # ===================

    def run_simulation(self):
        end_time = self.time_array[-1]

        while self.time < end_time:
            self._capture_time_snapshot()

            site = self.pick_random_site()
            rates = self.calculate_transition_rates(site)

            

            if not rates or sum(rates.values()) == 0:
                break  # Sin eventos posibles

            delta_t = (-np.log(np.random.rand()) / sum(rates.values())) / self.factor
            self.time += delta_t

            # print(f"\rtime: {self.time:.4f}, delta_t: {delta_t:.4e}", end="")
            # time.sleep(0.00000005)

            event = self.select_event(rates)
            self._apply_event(event, site)
            self._update_history(event)

        self._capture_remaining_snapshots()
        return self.occupancy

    def _apply_event(self, event, site):
        if event == "adsorption":
            self.lattice.lattice[site] = 1
        elif event == "desorption" and self.lattice.lattice[site] == 1:
            self.lattice.lattice[site] = 0
        elif event == "diffusion" and self.lattice.lattice[site] == 1:
            neighbors = self.lattice.get_neighbors(site)
            vacant = [n for n in neighbors if self.lattice.lattice[n] == 0]
            if vacant:
                new_site = vacant[np.random.randint(len(vacant))]
                self.lattice.lattice[site], self.lattice.lattice[new_site] = 0, 1

    def _update_history(self, event):
        self.events_count[event] += 1
        for key in self.events_history:
            self.events_history[key].append(self.events_count[key])

    def _capture_time_snapshot(self):
        for t in self.time_array:
            if self.occupancy[t] is None and self.time >= t:
                self.occupancy[t] = self.lattice.get_occupancy() * 100
                self.energies.append(self.calculate_total_energy())
                self.lattices.append(self.lattice.lattice.copy())

    def _capture_remaining_snapshots(self):
        for t in self.time_array:
            if self.occupancy[t] is None and self.time >= t:
                self.occupancy[t] = self.lattice.get_occupancy() * 100

    # ===================
    #     Utilidades
    # ===================

    def reset_lattice(self): 
        self.lattice.initialize()
        self.occupancy = {t: None for t in self.time_array}

    def plot_lattice(self, *args, **kwargs):
        self.lattice.plot(*args, **kwargs)


In [6]:
class KMCModelOptimizer:
    def __init__(self, exp_data, param_ranges, lattice_class, n_calls=200, n_simulations_avg=5):
        self.exp_data = np.array(exp_data)
        self.param_ranges = param_ranges
        self.lattice_class = lattice_class
        self.n_calls = n_calls
        self.n_simulations_avg = n_simulations_avg
        self.best_params = None
        self.best_error = float("inf")
        self.progress_bar = None
        self.time_array = np.array([0, 1, 3, 6, 9, 12, 20, 40, 60])  # o parametrizable

        if len(self.exp_data) != len(self.time_array):
            raise ValueError("La longitud de exp_data debe coincidir con la longitud de time_array.")

    def loss_function(self, params):
        param_dict = {name: val for name, val in zip(self.param_ranges.keys(), params)}

        processes = {}
        if 'e_ads' in param_dict:
            processes["adsorption"] = {
                "e_ads": param_dict["e_ads"],
                "e_int_abs": param_dict.get("e_int_abs", 0)
            }
        if 'e_des' in param_dict:
            processes["desorption"] = {
                "e_des": param_dict["e_des"],
                "e_int_des": param_dict.get("e_int_des", 0)
            }
        if 'e_diff' in param_dict:
            processes["diffusion"] = {
                "e_diff": param_dict['e_diff'],
                "e_int_diff": param_dict.get('e_int_diff', 0)
            }

        all_simulated_occupancies = []

        for _ in range(self.n_simulations_avg):
            lattice = self.lattice_class(int(param_dict['lattice_size']))
            kmc_model = KineticMonteCarlo(
                lattice=lattice,
                T=param_dict['T'],
                processes=processes,
                time_array=self.time_array
            )
            kmc_model.run_simulation()
            simulated_run_data = np.array([kmc_model.occupancy[t] for t in self.time_array])
            all_simulated_occupancies.append(simulated_run_data)

        if not all_simulated_occupancies:
            return float('inf')

        avg_simulated_data = np.mean(all_simulated_occupancies, axis=0)
        error = np.mean((avg_simulated_data - self.exp_data) ** 2)

        if error < self.best_error:
            self.best_error = error
            self.best_params = param_dict

        if self.progress_bar:
            self.progress_bar.set_postfix({
                "Mejor error": f"{self.best_error:.6f}",
                "Progreso": f"{self.progress_bar.n}/{self.progress_bar.total}"
            })
            self.progress_bar.update(1)

        return error

    def optimize(self):
        self.progress_bar = tqdm(
            total=self.n_calls,
            desc="🚀 Optimización Bayesiana",
            position=0,
            ncols=100,
            unit="eval"
        )

        space = []
        for param_name, prange in self.param_ranges.items():
            if param_name == 'lattice_size':
                space.append(Integer(prange[0], prange[1], name=param_name))
            else:
                space.append(Real(prange[0], prange[1], name=param_name))

        start_time = time.time()

        result = gp_minimize(
            func=self.loss_function,
            dimensions=space,
            n_calls=self.n_calls,
            random_state=42
        )
        self.progress_bar.close()

        end_time = time.time()
        print("\n✅ Optimización completada")
        if self.best_params:
            print("🔹 Parámetros óptimos encontrados:", self.best_params)
            print(f"🔹 Error mínimo (MSE) encontrado: {self.best_error:.6f}")
        else:
            print("⚠️ No se encontraron parámetros óptimos.")
            print("   Últimos parámetros:", {s.name: v for s, v in zip(space, result.x)})
            print(f"   Último error: {result.fun:.6f}")

        print(f"⏳ Tiempo total de ejecución: {end_time - start_time:.2f} segundos")
        self.run_kmc_with_best_params()

    def run_kmc_with_best_params(self):
        if not self.best_params:
            print("ℹ️ No se encontraron parámetros óptimos para ejecutar el modelo final.")
            return

        processes = {}
        if 'e_ads' in self.best_params:
            processes["adsorption"] = {
                "e_ads": self.best_params["e_ads"],
                "e_int_abs": self.best_params.get("e_int_abs", 0)
            }
        if 'e_des' in self.best_params:
            processes["desorption"] = {
                "e_des": self.best_params["e_des"],
                "e_int_des": self.best_params.get("e_int_des", 0)
            }
        if 'e_diff' in self.best_params:
            processes["diffusion"] = {
                "e_diff": self.best_params["e_diff"],
                "e_int_diff": self.best_params.get("e_int_diff", 0)
            }


        print(f"\n⚙️ Ejecutando simulación final con los mejores parámetros: {self.best_params}")
        lattice = self.lattice_class(int(self.best_params['lattice_size']))
        kmc_model = KineticMonteCarlo(
            lattice=lattice,
            T=self.best_params['T'],
            processes=processes,
            time_array=self.time_array
        )
        kmc_model.run_simulation()

        simulated_final_data = np.array([kmc_model.occupancy[t] for t in self.time_array])
        print("📊 Datos experimentales:", self.exp_data)
        print("📈 Datos simulados:", simulated_final_data)
        final_error = np.mean((simulated_final_data - self.exp_data) ** 2)
        print(f" MSE final (verificación): {final_error:.6f}")

In [7]:
fit_details = pd.read_csv("fit_details.csv")
fit_details


,Fuente(experimento),Texp0,Texp1,Texp2,Texp3,Texp4,Texp5,Texp6,Texp7,Texp8,...,MSE,best_error,e_ads,e_int_abs,e_des,e_int_des,e_diff,e_int_diff,lattice_size,T
0,Adsorcion de As+5 (60ppmSb-As),0,7.356582,12.689700,16.639759,18.501776,20.487569,23.818749,27.994834,30.050587,...,0.180722,0.102170,0.081506,0.042231,0.272000,0.068302,0.160000,0.160000,765,300.652977
1,Adsorcion de As+5 (60ppmAs-As),0,20.779935,25.251636,27.748749,29.965348,32.368957,36.389638,43.149442,48.682691,...,2.453615,1.915544,0.040199,0.042529,0.272000,0.068000,0.640000,0.108811,600,300.000000
2,Adsorcion de As+5 (90ppmAs-As),0,17.067494,22.581181,27.208246,29.441427,31.863017,35.913776,42.724149,48.187964,...,0.792748,0.661106,0.047140,0.038715,0.243444,0.267271,0.586739,0.150995,737,300.008465
3,Adsorcion de Pb (beta - FeOOH 0.05 M-Pb),0,1.145313,1.696100,2.264219,3.106472,3.699600,4.016499,4.056284,4.106165,...,0.135049,0.037399,0.115559,0.077452,0.099043,0.166115,0.160000,0.118336,783,300.000000
4,Adsorcion de Sb (beta - FeOOH 0.05 M-Sb),0,6.008902,10.187933,12.134026,13.930267,15.141939,16.922354,20.326410,23.376360,...,0.123435,0.109109,0.087042,0.051374,0.147142,0.149227,0.593463,0.085338,658,300.623917
5,Adsorcion de Sb (60ppmSb-Sb),0,27.241766,41.585839,46.084479,48.311079,49.992389,52.567519,55.551390,57.868871,...,3.111281,3.198456,0.034788,0.035238,0.250392,-0.027200,NaN,NaN,600,300.525227
6,Adsorcion de Sb (60ppmAs-Sb),0,34.587323,41.906065,44.300046,46.055632,46.967624,48.723210,51.983584,54.331965,...,1.153845,39.797460,0.001187,0.081499,0.234875,-0.015344,0.211515,-0.039092,315,300.025359
7,Adsorcion de Sb (60ppmHg-Sb),0,29.154068,36.910247,42.092633,43.724502,44.948404,50.227982,62.491001,64.050876,...,6.121742,5.697875,0.039200,0.032803,0.272000,0.148845,0.206789,0.160000,600,300.198780
8,Adsorcion de Sb (120ppmAs-Sb),0,21.817095,29.432051,34.577737,37.286592,39.562941,43.523788,51.331664,56.111996,...,1.028285,0.843838,0.039200,0.035252,0.272000,0.272000,0.640000,0.068713,727,300.644217
9,Adsorcion de Sb (120ppmSb-Sb),0,12.184766,16.710792,20.125645,22.100067,23.401391,26.923940,33.789545,37.715952,...,0.945446,0.596735,0.080085,0.040421,0.272000,0.111082,0.187424,0.160000,600,300.358782


In [8]:
exp_curves = fit_details.iloc[:, 1:10]
opt_energies = fit_details[['e_ads', 'e_int_abs', 'e_des', 'e_int_des', 'e_diff', 'e_int_diff']]
opt_temperatures = fit_details['T']
opt_sizes = fit_details['lattice_size']

time_array = np.array([0, 1, 3, 6, 9, 12, 20, 40, 60])


e_ads = opt_energies['e_ads'].tolist()
e_int_abs = opt_energies['e_int_abs'].tolist()
e_des = opt_energies['e_des'].tolist()
e_int_des = opt_energies['e_int_des'].tolist()
e_diff = opt_energies['e_diff'].tolist()
e_int_diff = opt_energies['e_int_diff'].tolist()
lattice_size_opt = opt_sizes.tolist()
T_opt = opt_temperatures.tolist()


e_diff

[0.16,
 0.6400000000000001,
 0.5867385173417683,
 0.16,
 0.5934632807795039,
 nan,
 0.2115146611413667,
 0.2067893372770648,
 0.6400000000000001,
 0.1874243043104213,
 0.6097390484280805,
 0.2032743983512453]

In [10]:
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.lines import Line2D
from matplotlib.colors import to_rgba
from itertools import cycle
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
import os
import scienceplots

# Estilo gráfico
plt.style.use(['science', 'ieee'])
mpl.rcParams['text.usetex'] = False
mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['font.serif'] = ['Times New Roman', 'DejaVu Serif', 'Computer Modern Roman']
mpl.rcParams['font.size'] = 13
mpl.rcParams['axes.labelsize'] = 16
mpl.rcParams['legend.fontsize'] = 12
mpl.rcParams['xtick.labelsize'] = 13
mpl.rcParams['ytick.labelsize'] = 13
mpl.rcParams['lines.linewidth'] = 1.5
mpl.rcParams['axes.linewidth'] = 0.8
mpl.rcParams['grid.linewidth'] = 0.5
mpl.rcParams.update({
    'pdf.fonttype': 42,
    'ps.fonttype': 42,
    'svg.fonttype': 'none'
})

# Paleta de colores
custom_palette = (
    list(plt.get_cmap("Dark2").colors) +
    list(plt.get_cmap("Set2").colors) +
    list(plt.get_cmap("Paired").colors)
)
color_cycle = cycle(custom_palette)

# Modelos
def pseudo_primer_orden(t, k1, qe):
    return qe * (1 - np.exp(-k1 * t))

def pseudo_segundo_orden(t, k2, qe):
    return (k2 * qe**2 * t) / (1 + k2 * qe * t)

def elovich(t, a, b):
    return (1 / b) * np.log(1 + a * b * t)


modelos = {
    "Pseudo-primer orden": pseudo_primer_orden,
    "Pseudo-segundo orden": pseudo_segundo_orden,
    "Elovich": elovich
}

def comparar_ajustes(t, qt_exp, qt_kmc, nombre_exp="experimento", output_dir="figuras_kmc"):
    os.makedirs(output_dir, exist_ok=True)
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8.5, 7.5), dpi=300, sharex=True)
    residuos_dict = {}
    color_map = {}
    color_cycle_local = cycle(custom_palette)

    # kMC
    color_kmc = next(color_cycle_local)
    ax1.scatter(t, qt_exp, color=to_rgba(color_kmc, alpha=0.35), marker='s', s=30, label="Datos experimentales")
    ax1.plot(t, qt_kmc, linestyle='--', marker='o', color=color_kmc, label='Modelo kMC')
    residuos_dict["Modelo kMC"] = qt_exp - qt_kmc
    color_map["Modelo kMC"] = color_kmc

    # Ajuste de modelos
    for nombre, modelo in modelos.items():
        try:
            if nombre == "Elovich":
                p0 = [1, 1]
                bounds = (0, [np.inf, np.inf])
            else:
                p0 = [0.1, max(qt_exp)]
                bounds = (0, [np.inf, np.inf])

            popt, _ = curve_fit(modelo, t, qt_exp, p0=p0, bounds=bounds, maxfev=20000)
            qt_modelo = modelo(t, *popt)

            color = next(color_cycle_local)
            ax1.plot(t, qt_modelo, label=nombre, linestyle='-', marker='o', markersize=4, color=color)
            residuos_dict[nombre] = qt_exp - qt_modelo
            color_map[nombre] = color

        except Exception as e:
            print(f"Error ajustando {nombre}: {e}")

    # Personalización gráfico superior
    ax1.set_ylabel("$Q_t$ (\%)")
    ax1.grid(True)
    ax1.legend(loc='best', frameon=False, fontsize=12)

    # Gráfico de residuos
    for nombre, resid in residuos_dict.items():
        ax2.plot(t, resid, label=nombre, marker='o', linestyle='-', color=color_map[nombre])

    ax2.axhline(0, linestyle='--', color='gray', lw=0.8)
    ax2.set_xlabel("Tiempo (min)")
    ax2.set_ylabel("Residuos ($Q_{exp} - Q_{modelo}$)")
    ax2.grid(True)

    plt.tight_layout()
    path_fig = os.path.join(output_dir, f"tradicional_vs_kmc_{nombre_exp}.pdf")
    fig.savefig(path_fig, bbox_inches="tight")
    plt.close(fig)
    print(f"Figura guardada en: {path_fig}")

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def extraer_errores_modelos(t, qt_exp, qt_kmc):
    errores = []
    n = len(t)

    # Modelo kMC (externo)
    mse_kmc = mean_squared_error(qt_exp, qt_kmc)
    r2_kmc = r2_score(qt_exp, qt_kmc)
    r2_adj_kmc = 1 - (1 - r2_kmc) * (n - 1) / (n - 1 - 1)  # p=1 (asume 1 parámetro para referencia)

    errores.append({
        "Modelo": "Modelo kMC",
        "MAE": mean_absolute_error(qt_exp, qt_kmc),
        "RMSE": np.sqrt(mse_kmc),
        "MSE": mse_kmc,
        "R²": r2_kmc,
        "R² ajustado": r2_adj_kmc
    })

    # Modelos cinéticos clásicos
    for nombre, modelo in modelos.items():
        try:
            if nombre == "Elovich":
                p0 = [1, 1]
                bounds = (0, [np.inf, np.inf])
            else:
                p0 = [0.1, max(qt_exp)]
                bounds = (0, [np.inf, np.inf])

            popt, _ = curve_fit(modelo, t, qt_exp, p0=p0, bounds=bounds, maxfev=20000)
            qt_modelo = modelo(t, *popt)
            p = len(popt)

            mse = mean_squared_error(qt_exp, qt_modelo)
            r2 = r2_score(qt_exp, qt_modelo)
            r2_adj = 1 - (1 - r2) * (n - 1) / (n - p - 1)

            errores.append({
                "Modelo": nombre,
                "MAE": mean_absolute_error(qt_exp, qt_modelo),
                "RMSE": np.sqrt(mse),
                "MSE": mse,
                "R²": r2,
                "R² ajustado": r2_adj
            })

        except Exception as e:
            print(f"Error ajustando modelo {nombre}: {e}")

    df_errores = pd.DataFrame(errores)[["Modelo", "MAE", "RMSE", "MSE", "R²", "R² ajustado"]].round(5)
    return df_errores




qt_kmc = fit_details.loc[0, fit_details.columns.str.startswith('mean_t')].values
time_array = np.array([0, 1, 3, 6, 9, 12, 20, 40, 60])
qt_datos = exp_curves.iloc[0, :9].to_numpy()
resultados = comparar_ajustes(time_array, qt_datos, qt_kmc)
errores = extraer_errores_modelos(time_array, qt_datos, qt_kmc)
errores

Figura guardada en: figuras_kmc/tradicional_vs_kmc_experimento.pdf


,Modelo,MAE,RMSE,MSE,R²,R² ajustado
0,Modelo kMC,0.35821,0.42511,0.18072,0.99783,0.99752
1,Pseudo-primer orden,1.81001,2.14393,4.59642,0.94488,0.92651
2,Pseudo-segundo orden,0.99683,1.20697,1.45677,0.98253,0.97671
3,Elovich,0.13761,0.17727,0.03142,0.99962,0.99950
